# Extraction Azure API Test

## Extraction Azure APIM

### Install the required packages

In [ ]:
#%pip install python-dotenv
#%pip install requests
#%pip install azure-storage-blob
#%pip install azure-ai-documentintelligence==1.0.0b4
#%pip install tabulate

### Load Configuration Settings

In [ ]:
from dotenv import dotenv_values,load_dotenv
import sys
sys.path.append('../common')
from helper import Helper

# Load the environment variables
load_dotenv()

# Load the secrets from the environment file
secrets = dotenv_values("secrets.env") # Modify environment file name to match respective environment file

# Load APIM Environment Variables
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
region_override = secrets.get("REGION_OVERRIDE")

helper = Helper(environment, "extraction/azure", api_key, region_override)

### Initialize Extraction Service API Service Inputs

#### Set ADI API Version

In [ ]:
# Must use the latest API version 2024-07-31-preview.
api_version = "2024-07-31-preview"

# Validate API version input
assert api_version == "2024-07-31-preview", "Invalid api version. Must use the latest API version 2024-07-31-preview for API compatibility."

#### Set Data Source Variables

In [ ]:
# Obtain Container Name found in APIM Storage Accounts and set as value for source_container_name
source_container_name = "reg-source"

### List all available blobs (files)

In [ ]:
from azure.storage.blob import BlobServiceClient
from tabulate import tabulate

# List all the blobs in the container
def list_containers_blobs(account_connection_string, source_container_name):
    blob_service_client = BlobServiceClient.from_connection_string(account_connection_string)

    files = blob_service_client.get_container_client(source_container_name).list_blobs()

    blob_names = []
    for file in files:
        if file.name not in ["None", "sourceData"]:
            blob_names.append(file.name)

    return blob_names

helper.log('Retrieving blobs from container: ' + source_container_name)

# Get the list of blobs in the container
containerBlobList = list_containers_blobs(account_connection_string, source_container_name)

# Prepare data for the table
table_data = [[blob_name] for blob_name in containerBlobList]

# Define table headers
headers = ["File Name"]

# Print the table
helper.log(tabulate(table_data, headers, tablefmt="grid"))

# Validate output exists
assert containerBlobList is not None

### Set target blob and output format

In [ ]:
# Copy one of the above file name and set as value to the variable blob_name
# Select the first blob whose name starts with "sourceData/"
blob_name = next((b for b in containerBlobList if b.startswith("sourceData/")), None)
assert blob_name is not None, "No blob starting with 'sourceData/' found in the container. Please verify the container contents and ensure it is properly populated with the required blobs."


helper.log("Blob: " + blob_name)

# Supported values are "text" and "markdown"
output_format = "markdown"

# Validate blob_name selected is available in the source container and output_format is valid
assert blob_name in containerBlobList, "Provided Blob Name is not available in the source container. Please select a valid blob name from the list above."
assert output_format in ["text", "markdown"], "Invalid output format. Supported values are 'text' and 'markdown'."

### Generate a SAS token for the target blob

In [ ]:
from azure.storage.blob import generate_blob_sas, BlobSasPermissions, BlobServiceClient
from datetime import datetime, timedelta, timezone
from urllib.parse import quote

# Generate SAS token for the specified blob
def generate_sas_token(account_name, source_container_name, blob_name, account_key):
    sas_token = generate_blob_sas(
        account_name=account_name,
        container_name=source_container_name,
        blob_name=blob_name,
        account_key=account_key,
        permission=BlobSasPermissions(read=True),
        expiry=datetime.now(timezone.utc) + timedelta(hours=1)
    )
    return sas_token

# Generate SAS URL for the specified blob
def generate_sas_for_blob(account_connection_string, source_container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(account_connection_string)
    encoded_blob_name = quote(blob_name)
    sas_token = generate_sas_token(blob_service_client.account_name, source_container_name, blob_name, blob_service_client.credential.account_key)
    sas_url = f"https://{blob_service_client.account_name}.blob.core.windows.net/{source_container_name}/{encoded_blob_name}?{sas_token}"

    helper.log(sas_url)  # Prints the SAS URL for the specified blob

    return sas_url


# Generate SAS URL for the specified blob and store it in a variable
source_sas_url = generate_sas_for_blob(account_connection_string, source_container_name, blob_name)

### List all available extraction models

In [ ]:
from azure.ai.documentintelligence import DocumentIntelligenceAdministrationClient
from azure.core.credentials import AzureKeyCredential
from azure.core.pipeline.policies import HeadersPolicy

# Create an instance of the Document Intelligence Administration Client
adiAdminClient = DocumentIntelligenceAdministrationClient(
    endpoint=F"https://{environment}/extraction/azure/",
    credential=AzureKeyCredential(api_key),
    api_version=api_version,
    headers_policy=HeadersPolicy({
        "Ocp-Apim-Subscription-Key": api_key,
        "x-kpmg-charge-code": '803000015283',
        "x-kpmg-region-override": region_override,
    }))

# List document models
modelsListing = adiAdminClient.list_models()

# Prepare data for the table
table_data = []
for model in modelsListing:
    table_data.append([model.model_id, model.description])

# Define table headers
headers = ["Model ID", "Description"]

# Print the table
helper.log(tabulate(table_data, headers, tablefmt="grid"))

# Validate output exists
assert DocumentIntelligenceAdministrationClient is not None

### Get Model

In [ ]:
# Set preferred model for extraction based on available models in previous step
modelId = "prebuilt-layout"

In [ ]:
# Get model info for specific model
modelinfo = adiAdminClient.get_model(model_id=modelId)
helper.log(f"Model ID: {modelinfo.model_id}\nModel Description: {modelinfo.description}")

# Validate output exists
assert modelinfo.model_id == modelId

### Analyze Specified Document

In [ ]:
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest, ContentFormat
from azure.ai.documentintelligence._operations._patch import AnalyzeDocumentLROPoller
from azure.ai.documentintelligence.models import OperationStatus

# Create an instance of the Document Intelligence Client
adiClient = DocumentIntelligenceClient(
    endpoint=f"https://{environment}/extraction/azure/",
    credential=AzureKeyCredential(api_key),
    api_version=api_version,
    headers_policy=HeadersPolicy({
        "Ocp-Apim-Subscription-Key": api_key,
        "x-kpmg-charge-code": '803000015283',
        "x-kpmg-region-override": region_override,
    }))

# Analyze the document
poller:AnalyzeDocumentLROPoller = adiClient.begin_analyze_document(
    model_id = modelId,
    analyze_request = AnalyzeDocumentRequest(
        url_source = source_sas_url
    ),
    output_content_format=ContentFormat(output_format)
)

# Print Progress
helper.log("Analyzing document...")

# Get the result
result = poller.result()

assert poller.status() == OperationStatus.SUCCEEDED

### Get Analyze Result

In [ ]:
from IPython.display import display, Markdown

# Display Text or Markdown output based on the specified format
display(Markdown(result.content))

# Validate output exists
assert result.content_format == output_format, "Output format does not match the specified format. Please check the output format or modelID and try again."